In [61]:
#pip install selenium

In [62]:
#pip install undetected-chromedriver

In [63]:
import selenium
import re
import requests
from bs4 import BeautifulSoup

In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import undetected_chromedriver as uc
import random

## BASE COMPETIDORES

In [65]:
#data.to_csv('data_web_scrap.csv', index=False) 

In [66]:
compes = pd.read_excel("Competidores.xlsx", sheet_name = 'Limpios')

In [67]:
n = compes.shape[0]
competidores = []

for i in range(0,n):
    c = "+".join(compes['COMPETIDOR'][i].split())
    s = "+".join(compes['SUCURSAL'][i].split())
    comp_suc = c+'+'+s
    competidores.append(comp_suc)

competidores = pd.DataFrame(competidores, columns=['competidores'])

In [68]:
competidores['COD_PAIS'] = compes['COD_PAIS']
competidores['COD_COMPETIDOR'] = compes['COD_COMPETIDOR']
competidores['COMPETIDOR'] = compes['COMPETIDOR']
competidores['COD_SUCURSAL'] = compes['COD_SUCURSAL']
competidores['SUCURSAL'] = compes['SUCURSAL']
competidores['PAIS+COMPETIDOR'] = competidores['COD_PAIS'].astype(str) + '+' + competidores['COD_COMPETIDOR'].astype(str) + '+' + competidores['COMPETIDOR']

In [69]:
competidores

,competidores,COD_PAIS,COD_COMPETIDOR,COMPETIDOR,COD_SUCURSAL,SUCURSAL,PAIS+COMPETIDOR
0,PriceSmart+Escazu,1,10100,PriceSmart,10101,Escazu,1+10100+PriceSmart
1,PriceSmart+Tibas,1,10100,PriceSmart,10102,Tibas,1+10100+PriceSmart
2,PriceSmart+Zapote,1,10100,PriceSmart,10103,Zapote,1+10100+PriceSmart
3,PriceSmart+Tres+Rios,1,10100,PriceSmart,10104,Tres Rios,1+10100+PriceSmart
4,PriceSmart+Santa+Ana,1,10100,PriceSmart,10105,Santa Ana,1+10100+PriceSmart
...,...,...,...,...,...,...,...
1271,Super+La+Suiza+La+Suiza,1,106600,Super La Suiza,106601,La Suiza,1+106600+Super La Suiza
1272,Super+Mei+Super+Mei,1,106700,Super Mei,106701,Super Mei,1+106700+Super Mei
1273,Supermercado+El+Ahora+Supermercado+El+Ahora,1,106800,Supermercado El Ahora,106801,Supermercado El Ahora,1+106800+Supermercado El Ahora
1274,Super+Mercado+El+Arbolito+Super+Mercado+El+Arb...,1,106900,Super Mercado El Arbolito,106901,Super Mercado El Arbolito,1+106900+Super Mercado El Arbolito


## SCRAPPER

In [70]:
initText1 = 'data-hour="'
endText1 = '"'

initText2 = '"height:'
endText2 = 'px;"'

diccionario = dict(
    cod_pais=[],
    cod_competidor=[],
    competidor=[],
    comp_suc=[],
    cod_sucursal=[],
    sucursal=[],
    dia=[],
    hora=[],
    pixeles=[]
)

In [71]:
from datetime import datetime

print(datetime.now())

2024-07-18 08:33:59.159136


In [72]:
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = option)

n = competidores.shape[0]

for m in range(0, n):
    driver.get("https://www.google.com/search?q=" + competidores.iloc[m, 0])
    dias = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
    
    time.sleep(random.randint(5,10))
    
    # Validacion existencia grafico
    html = driver.page_source
    if ((html.find('Horario de mayor concurrencia') != -1) & (html.find('data-hour="10"') != -1)):

        for j in dias:
            path_dias = '//*[@class="pptUAb" and @aria-label="' + str(j) + '"]'
            WebDriverWait(driver, 5)\
                .until(EC.element_to_be_clickable((By.XPATH, path_dias)))\
                .click()
            
            # Validacion si esta cerrado
            path_validar = '//*[@class="W1cfjc"]'
            
            elemento2 = driver.find_element("xpath", path_validar)
            texto2 = elemento2.get_attribute("outerHTML")
    
            if (texto2.find('Cerrado') == -1):
                # Obtener horas del dia
                html2 = driver.find_element("xpath", '//*[@class="tcuWYc"]')
                texto_html2 = html2.get_attribute("outerHTML")
                horas = re.findall(initText1 + '(.+?)' + '"', texto_html2)
                horas = list(map(int, horas))
        
                # Obteniendo los pixeles
                for i in horas:
                    path_horas = '//*[@class="xiXXie" and @data-hour="' + str(i) + '"]'
                    
                    elemento = driver.find_element("xpath", path_horas)
                    texto = elemento.get_attribute("outerHTML")
                
                    hora = re.search(initText1 + '(.+?)' + endText1, texto).group(1)
                    pixeles = re.search(initText2 + '(.+?)' + endText2, texto).group(1)

                    diccionario["hora"].append(int(hora))
                    diccionario["pixeles"].append(float(pixeles))
                    diccionario["dia"].append(j)
                    diccionario["comp_suc"].append(competidores.iloc[m, 0])
                    diccionario["competidor"].append(competidores.iloc[m, 3])
                    diccionario["cod_competidor"].append(competidores.iloc[m, 2])
                    diccionario["cod_sucursal"].append(competidores.iloc[m, 4])
                    diccionario["sucursal"].append(competidores.iloc[m, 5])
                    diccionario["cod_pais"].append(competidores.iloc[m, 1])
            #driver.quit()
            else:
                diccionario["hora"].append("Cerrado")
                diccionario["pixeles"].append("")
                diccionario["dia"].append(j)
                diccionario["comp_suc"].append(competidores.iloc[m, 0])
                diccionario["competidor"].append(competidores.iloc[m, 3])
                diccionario["cod_competidor"].append(competidores.iloc[m, 2])
                diccionario["cod_sucursal"].append(competidores.iloc[m, 4])
                diccionario["sucursal"].append(competidores.iloc[m, 5])
                diccionario["cod_pais"].append(competidores.iloc[m, 1])
                
        
    else:
        diccionario["hora"].append("")
        diccionario["pixeles"].append("")
        diccionario["dia"].append("Sin grafico")
        diccionario["comp_suc"].append(competidores.iloc[m, 0])
        diccionario["competidor"].append(competidores.iloc[m, 3])
        diccionario["cod_competidor"].append(competidores.iloc[m, 2])
        diccionario["cod_sucursal"].append(competidores.iloc[m, 4])
        diccionario["sucursal"].append(competidores.iloc[m, 5])
        diccionario["cod_pais"].append(competidores.iloc[m, 1])
        

In [73]:
print(datetime.now())

2024-07-18 12:37:42.135470


In [74]:
data = pd.DataFrame(diccionario)

In [75]:
data

,cod_pais,cod_competidor,competidor,comp_suc,cod_sucursal,sucursal,dia,hora,pixeles
0,1,10100,PriceSmart,PriceSmart+Escazu,10101,Escazu,lunes,6,0.0
1,1,10100,PriceSmart,PriceSmart+Escazu,10101,Escazu,lunes,7,0.0
2,1,10100,PriceSmart,PriceSmart+Escazu,10101,Escazu,lunes,8,0.0
3,1,10100,PriceSmart,PriceSmart+Escazu,10101,Escazu,lunes,9,14.25
4,1,10100,PriceSmart,PriceSmart+Escazu,10101,Escazu,lunes,10,24.75
...,...,...,...,...,...,...,...,...,...
75666,1,107000,Super Mercado Fuente Termal,Super+Mercado+Fuente+Termal+Super+Mercado+Fuen...,107001,Super Mercado Fuente Termal,domingo,19,30.75
75667,1,107000,Super Mercado Fuente Termal,Super+Mercado+Fuente+Termal+Super+Mercado+Fuen...,107001,Super Mercado Fuente Termal,domingo,20,21.75
75668,1,107000,Super Mercado Fuente Termal,Super+Mercado+Fuente+Termal+Super+Mercado+Fuen...,107001,Super Mercado Fuente Termal,domingo,21,0.0
75669,1,107000,Super Mercado Fuente Termal,Super+Mercado+Fuente+Termal+Super+Mercado+Fuen...,107001,Super Mercado Fuente Termal,domingo,22,0.0


### Limpieza

In [76]:
data2 = data[data['dia'] != 'Sin grafico']

In [77]:
data3 = data2[data2['hora'] != 'Cerrado']

data_completa = data3.set_index(["competidor"]).join(
    competidores.set_index(["competidores"]),
    how="left").drop_duplicates()

data_completa['compe+suc'] = data_completa.index.tolist()

data_completa2 = data_completa.reset_index()

### Pesos por sucursales

In [78]:
a = data3.groupby(['cod_sucursal', 'dia'])["pixeles"].sum()

In [79]:
b = data3.groupby(['cod_sucursal'])["pixeles"].sum()

In [80]:
# Peso por dia de sucursales
c = a.div(b, level='cod_sucursal')

In [81]:
c

cod_sucursal  dia      
10101         domingo      0.187358
              jueves       0.119827
              lunes        0.116417
              martes       0.108913
              miércoles    0.114143
                             ...   
107001        lunes        0.136585
              martes       0.138211
              miércoles    0.143089
              sábado        0.15411
              viernes      0.173442
Name: pixeles, Length: 4151, dtype: object

#### Formato a tabla

In [82]:
pesos_suc = pd.DataFrame(c).reset_index()

In [83]:
pesos_suc2 = pesos_suc.set_index(["cod_sucursal"]).join(
    competidores.set_index(["COD_SUCURSAL"]),
    how="left").drop_duplicates()

In [84]:
pesos_suc2 = pesos_suc2.reset_index()
pesos_suc2 = pesos_suc2.drop(['PAIS+COMPETIDOR'], axis = 1)

In [85]:
pesos_suc3 = pesos_suc2.loc[:, ['COD_PAIS', 'COD_COMPETIDOR', 'COMPETIDOR', 'index', 'SUCURSAL', 'dia', 'pixeles']]

In [86]:
pesos_suc3 = pesos_suc3.rename(columns={"dia": "DIA", "pixeles": "PESOS", 'index': 'COD_SUCURSAL'})
pesos_suc = pesos_suc3

In [87]:
# salida sucursal
pesos_suc

,COD_PAIS,COD_COMPETIDOR,COMPETIDOR,COD_SUCURSAL,SUCURSAL,DIA,PESOS
0,1,10100,PriceSmart,10101,Escazu,domingo,0.187358
1,1,10100,PriceSmart,10101,Escazu,jueves,0.119827
2,1,10100,PriceSmart,10101,Escazu,lunes,0.116417
3,1,10100,PriceSmart,10101,Escazu,martes,0.108913
4,1,10100,PriceSmart,10101,Escazu,miércoles,0.114143
...,...,...,...,...,...,...,...
4076,1,107000,Super Mercado Fuente Termal,107001,Super Mercado Fuente Termal,lunes,0.136585
4077,1,107000,Super Mercado Fuente Termal,107001,Super Mercado Fuente Termal,martes,0.138211
4078,1,107000,Super Mercado Fuente Termal,107001,Super Mercado Fuente Termal,miércoles,0.143089
4079,1,107000,Super Mercado Fuente Termal,107001,Super Mercado Fuente Termal,sábado,0.15411


### Pesos por competidor pais

In [88]:
data3['PAIS+COMPETIDOR'] = data3['cod_pais'].astype(str) + '+' + data3['cod_competidor'].astype(str) + '+' + data3['competidor']

C:\Users\Elda Calderon\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
d = data3.groupby(['PAIS+COMPETIDOR', 'dia'])["pixeles"].sum()

In [90]:
e = data3.groupby(['PAIS+COMPETIDOR'])["pixeles"].sum()

In [91]:
# Peso por dia de competidores
f = d.div(e, level='PAIS+COMPETIDOR')

#### Formato a tabla

In [92]:
pesos_comp = pd.DataFrame(f).reset_index()

In [93]:
pesos_comp2 = pd.DataFrame(pesos_comp["PAIS+COMPETIDOR"].str.split("+", expand=True))
pesos_comp2.columns = ['COD_PAIS', 'COD_COMPETIDOR', 'COMPETIDOR']

In [94]:
pesos_comp3 = pd.concat([pesos_comp2, pesos_comp], axis = 1)

In [95]:
pesos_comp3['COD_SUCURSAL'] = ''
pesos_comp3['SUCURSAL'] = ''
pesos_comp4 = pesos_comp3.drop('PAIS+COMPETIDOR', axis = 1)

In [96]:
pesos_comp4 = pesos_comp4.loc[:, ['COD_PAIS', 'COD_COMPETIDOR', 'COMPETIDOR', 'COD_SUCURSAL', 'SUCURSAL', 'dia', 'pixeles']]

In [97]:
pesos_comp4 = pesos_comp4.rename(columns={"dia": "DIA", "pixeles": "PESOS"})
pesos_comp = pesos_comp4

In [98]:
# salida competidor
pesos_comp

,COD_PAIS,COD_COMPETIDOR,COMPETIDOR,COD_SUCURSAL,SUCURSAL,DIA,PESOS
0,1,100100,La Troja,,,domingo,0.023145
1,1,100100,La Troja,,,jueves,0.15581
2,1,100100,La Troja,,,lunes,0.141255
3,1,100100,La Troja,,,martes,0.192555
4,1,100100,La Troja,,,miércoles,0.122882
...,...,...,...,...,...,...,...
1019,5,54300,Super Alameda,,,lunes,0.137506
1020,5,54300,Super Alameda,,,martes,0.136857
1021,5,54300,Super Alameda,,,miércoles,0.137506
1022,5,54300,Super Alameda,,,sábado,0.16118


### Pesos por pais

In [99]:
g = data3.groupby(['cod_pais', 'dia'])["pixeles"].sum()

In [100]:
h = data3.groupby(['cod_pais'])["pixeles"].sum()

In [101]:
# Peso por dia de competidores
i = g.div(h, level='cod_pais')

#### Formato a tabla

In [102]:
pesos_pais = pd.DataFrame(i).reset_index()

In [103]:
pesos_pais['COD_SUCURSAL'] = '999' + pesos_pais['cod_pais'].astype(str) 
pesos_pais['SUCURSAL'] = 'DUMMY'
pesos_pais['COD_COMPETIDOR'] = '999' + pesos_pais['cod_pais'].astype(str) 
pesos_pais['COMPETIDOR'] = 'DUMMY'

In [104]:
pesos_pais = pesos_pais.loc[:, ['cod_pais', 'COD_COMPETIDOR', 'COMPETIDOR', 'COD_SUCURSAL', 'SUCURSAL', 'dia', 'pixeles']]

In [105]:
pesos_pais = pesos_pais.rename(columns={"dia": "DIA", "pixeles": "PESOS", 'cod_pais': 'COD_PAIS'})

In [106]:
# salida pais
pesos_pais

,COD_PAIS,COD_COMPETIDOR,COMPETIDOR,COD_SUCURSAL,SUCURSAL,DIA,PESOS
0,1,9991,DUMMY,9991,DUMMY,domingo,0.138454
1,1,9991,DUMMY,9991,DUMMY,jueves,0.133951
2,1,9991,DUMMY,9991,DUMMY,lunes,0.13489
3,1,9991,DUMMY,9991,DUMMY,martes,0.129989
4,1,9991,DUMMY,9991,DUMMY,miércoles,0.133121
5,1,9991,DUMMY,9991,DUMMY,sábado,0.174134
6,1,9991,DUMMY,9991,DUMMY,viernes,0.15546
7,2,9992,DUMMY,9992,DUMMY,domingo,0.146269
8,2,9992,DUMMY,9992,DUMMY,jueves,0.135361
9,2,9992,DUMMY,9992,DUMMY,lunes,0.135875


# Salida Final

In [107]:
salida = pd.concat([pesos_suc, pesos_comp, pesos_pais], axis = 0)

In [108]:
mes = str(datetime.now().strftime("%b"))
ano = str(datetime.now().strftime("%y"))

In [109]:
salida.to_csv('data_web_scrap_' + mes + ano + '.csv', index=False) 

In [111]:
data.to_csv('data_web_scrap_cruda_' + mes + ano + '.csv', index=False) 

## Con UC

In [110]:
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = option)

n = competidores.shape[0]

for m in range(0, n):
    browser.get("https://www.google.com/search?q=" + competidores.iloc[m, 0])
    dias = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
    
    time.sleep(random.randint(5,10))
    
    # Validacion existencia grafico
    html = driver.page_source
    if (html.find('Horario de mayor concurrencia') != -1):

        for j in dias:
            path_dias = '//*[@class="pptUAb" and @aria-label="' + str(j) + '"]'
            WebDriverWait(driver, 5)\
                .until(EC.element_to_be_clickable((By.XPATH, path_dias)))\
                .click()


            for i in range(6, 24):
                path_horas = '//*[@class="xiXXie" and @data-hour="' + str(i) + '"]'

                elemento = driver.find_element("xpath", path_horas)
                texto = elemento.get_attribute("outerHTML")

                hora = re.search(initText1 + '(.+?)' + endText1, texto).group(1)
                pixeles = re.search(initText2 + '(.+?)' + endText2, texto).group(1)

                diccionario["hora"].append(int(hora))
                diccionario["pixeles"].append(float(pixeles))
                diccionario["dia"].append(j)
                diccionario["competidor"].append(competidores.iloc[m, 0])
        #driver.quit()
        
    else:
        diccionario["hora"].append("")
        diccionario["pixeles"].append("")
        diccionario["dia"].append("Sin grafico")
        diccionario["competidor"].append(competidores.iloc[m, 0])


NameError: name 'browser' is not defined

Validar barra de hora actual

In [ ]:
initText1 = 'data-hour="'
endText1 = '"'

initText2 = '"height:'
endText2 = 'px;"'

diccionario = dict(
    dia=[],
    hora=[],
    pixeles=[]
)

In [ ]:
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = option)


driver.get("https://www.google.com/search?q=" + 'walmart+escazu')
dias = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
    
time.sleep(random.randint(5,10))
    
    
for j in dias:
    path_dias = '//*[@class="pptUAb" and @aria-label="' + str(j) + '"]'
    WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.XPATH, path_dias)))\
    .click()
            
    # Validacion si esta cerrado
    path_validar = '//*[@class="W1cfjc"]'
            
    elemento2 = driver.find_element("xpath", path_validar)
    texto2 = elemento2.get_attribute("outerHTML")
    
    if (texto2.find('Cerrado') == -1):
    # Obtener horas del dia
        html2 = driver.find_element("xpath", '//*[@class="tcuWYc"]')
        texto_html2 = html2.get_attribute("outerHTML")
        horas = re.findall(initText1 + '(.+?)' + '"', texto_html2)
        horas = list(map(int, horas))
        
    # Obteniendo los pixeles
    for i in horas:
        path_horas = '//*[@class="xiXXie" and @data-hour="' + str(i) + '"]'

        elemento = driver.find_element("xpath", path_horas)
        texto = elemento.get_attribute("outerHTML")

        hora = re.search(initText1 + '(.+?)' + endText1, texto).group(1)
        pixeles = re.search(initText2 + '(.+?)' + endText2, texto).group(1)

        diccionario["hora"].append(int(hora))
        diccionario["pixeles"].append(float(pixeles))
        diccionario["dia"].append(j)
        
           
        

In [ ]:
d = pd.DataFrame(diccionario)

In [ ]:
d[d['dia'] == 'jueves']